In [6]:
import numpy as np
import pandas as pd
import yfinance as yf

def get_returns(ticker, start, end):
    #call get_returns and store in a variable, or put into the other functions
    '''Takes data and performs yfinance query. Returns Adjusted Close price.
       _______________________________________________________________________
       
       ticker: The ticker of the stock.
       
       start: Download start date string (YYYY-MM-DD) or _datetime.
              Default is 1900-01-01.
       
       end: Download end date string (YYYY-MM-DD) or _datetime.
            Default is now.
    '''
    return yf.download(ticker, start=start, end=end)['Adj Close']

def get_pct_returns(ticker, start, end):
    '''Calculates percentage returns based on input.
       ________________________________________________
       
       ticker: The ticker of the stock.
       
       start: Download start date string (YYYY-MM-DD) or _datetime.
              Default is 1900-01-01.
       
       end: Download end date string (YYYY-MM-DD) or _datetime.
            Default is now.
    '''    
    #yf query, then picking AdjClose for Adjusted returns
    #then running pct change + dropna to give percentage change & remove NaN rows
    return yf.download(ticker, start=start, end=end)['Adj Close'].pct_change().dropna()

def path_simul_bootstr(returns, days, I0, seed, sims = 10000):
    ''' Simulates paths of daily returns over time, using the bootstrap method.
        
        returns: A dataframe or array of returns over time. The programme is designed 
                 to deal with daily data, but if the user wishes to use other time
                 periods, they can use the resample method.
        
        days: Time period over which the calculation is performed.
        
        I0: The investment multiple, or the original investment.
        
        sims: The number of simulations desired. Default is 10,000.
        
        seed: The seed for the randomised returns generated in each simulation.
              Use the same value if you wish to replicate results.   
    '''
    
    days = int(days)
    
    np.random.seed(seed)
    #random sampling w np.random.choice
    ret = np.random.choice(returns, size = days * sims, replace = True).reshape(sims, days)
    
    paths = (ret + 1).cumprod(axis = 1) * I0
    paths = np.hstack((np.ones(sims).reshape(sims, 1)*I0, paths))
    return paths

def var_bootstr(returns, days, prob, I0, seed, sims = 10000):
    '''Calculates expected Value at Risk for a given investment, using the bootstrap method.
       Returns are assumed
        
       returns: A dataframe or array of returns over time. The programme is designed 
                to deal with daily data, but if the user wishes to use other time
                periods, they can use the resample method.
        
       days: Time period over which the calculation is performed.
        
       I0: The investment multiple, or the original investment.
        
       sims: The number of simulations desired. Default is 10,000.
        
       seed: The seed for the randomised returns generated in each simulation.
             Use the same value if you wish to replicate results.   
    '''
    days = int(days)
    
    np.random.seed(seed)
    ret = np.random.choice(daily_returns, size = days * sims, replace = True).reshape(sims, days)
    
    paths = (ret + 1).prod(axis = 1) * I0
    #using conditional value at risk for more accurate estimate 
    final_p = paths[:, -1]
    var = final_p[final_p < np.percentile(final_p, prob*100)] - I0
    
    return var 